## Sprint 4 – Análisis de Slices Problemáticos del modelo BETO

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, f1_score, confusion_matrix


In [4]:
import os
from pathlib import Path

print("CWD (carpeta actual):", os.getcwd())
print("\nContenido de la carpeta actual:")
print(os.listdir())

print("\nBuscando carpetas de modelos con config.json...\n")
for p in Path(".").rglob("config.json"):
    print("->", p.parent)


CWD (carpeta actual): d:\_ JCAM 2010-2028\_3_MASTERAI_UNI\Ciclo4\Tesis 2 Proyecto\ProyectoClasificador\ClasificadorsemanticoBETO\notebooks

Contenido de la carpeta actual:
['1_depurando_dataset.ipynb', '2_baseline_tfidf_lr_cv.ipynb', '3_fine_tuning_beto_kfold.ipynb', '4_analisis_resultados_beto.ipynb', '5_inferencia_beto.ipynb', '6_slices_analisis_sprint4.ipynb', 'accuracy_por_epoca.png', 'beto_cls_best', 'beto_cls_best_clean', 'beto_cls_runs', 'beto_cls_runs_clean', 'modelo_beto_gpu', 'models', 'results_beto', '_Ex']

Buscando carpetas de modelos con config.json...

-> beto_cls_best
-> beto_cls_best_clean
-> beto_cls_runs\checkpoint-100
-> beto_cls_runs\checkpoint-150
-> beto_cls_runs\checkpoint-50
-> beto_cls_runs_clean\checkpoint-100
-> beto_cls_runs_clean\checkpoint-150
-> beto_cls_runs_clean\checkpoint-50
-> modelo_beto_gpu
-> models\beto_fold0_final
-> results_beto\checkpoint-400
-> results_beto\checkpoint-500


In [24]:
#Celda 4
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# 1. Ruta del modelo entrenado (relativa a la carpeta "notebooks")
MODEL_DIR = r"models\beto_fold0_final"

# 2. Cargar tokenizer y modelo en modo local (sin ir a HuggingFace)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
model.eval()

print("✅ Modelo cargado desde:", MODEL_DIR)


✅ Modelo cargado desde: models\beto_fold0_final


In [25]:
#Celda 4
import pandas as pd

# AJUSTA ESTA RUTA A TU CSV REAL DE EVALUACIÓN
#df = pd.read_csv(r"..\data\processed\clean_v2\folds\corpus_clean_v2_folds.csv")
df = pd.read_csv(r"..\data\processed\clean_v2\folds\val_fold0.csv")

df.head()


,text,labels
0,Habla de verdad y solo cuenta su parte,0
1,La fuerza del pueblo es más fuerte que cualqui...,3
2,"Se dice ejemplo, pero su historia está llena d...",0
3,"Ellos confunden crítica con traición, nosotros...",1
4,"Promete educación, pero no lee ni los informes",0


In [21]:
slice_ultrashort = df_pred[df_pred["num_tokens"] <= 5]

from sklearn.metrics import f1_score

f1_ultrashort = f1_score(slice_ultrashort["label_true"], slice_ultrashort["label_pred"], average="macro")

print("Slice: textos ultracortos (<=5 palabras)")
print("F1:", f1_ultrashort)
print("Tamaño:", len(slice_ultrashort))


Slice: textos ultracortos (<=5 palabras)
F1: 1.0
Tamaño: 9


In [22]:
person_terms = [" yo ", " me ", " mi ", " nosotros ", " nos "]

df_pred["has_1p"] = df_pred["text"].str.lower().apply(lambda x: any(t in x for t in person_terms))

slice_1p = df_pred[df_pred["has_1p"] == True]

from sklearn.metrics import f1_score
f1_1p = f1_score(slice_1p["label_true"], slice_1p["label_pred"], average="macro")

print("Slice: frases con primera persona (yo/me/mi/nosotros/nos)")
print("F1:", f1_1p)
print("Tamaño:", len(slice_1p))


Slice: frases con primera persona (yo/me/mi/nosotros/nos)
F1: 1.0
Tamaño: 39


Tabla resumen de los 2 slides

In [27]:
resumen_slices = pd.DataFrame([
    {
        "Slice": "Textos ultracortos (<=5 palabras)",
        "F1": 1.0,                # aquí puedes poner f1_ultrashort si lo guardaste
        "Tamano": 9               # o len(slice_ultrashort) si lo tienes
    },
    {
        "Slice": "Frases con primera persona",
        "F1": f1_1p,
        "Tamano": len(slice_1p)
    }
])

resumen_slices


,Slice,F1,Tamano
0,Textos ultracortos (<=5 palabras),1.0,9
1,Frases con primera persona,1.0,39
